In [1]:
import pydrake.solvers.mathematicalprogram as mp
import numpy as np

In [2]:
from polynomial import Polynomial
from piecewise import Piecewise

In [5]:
def piecewise_polynomial_variable(prog, domain, dimension, degree):
    C = [prog.NewContinuousVariables(degree + 1, len(domain) - 1) for _ in range(dimension)]
    C = np.stack(C, axis=2)
    assert C.shape == (degree + 1, len(domain) - 1, dimension)
    return Piecewise(domain,
                     [Polynomial([C[i, j, :] for i in range(degree + 1)]) 
                          for j in range(len(domain) - 1)])

In [13]:
def require_continuity(prog, piecewise):
    for t in piecewise.breaks[1:-1]:
        frombelow = piecewise.from_below(t)
        fromabove = piecewise.from_above(t)
        for i in range(frombelow.size):
            print "continuity:", frombelow.flat[i].Expand(), fromabove.flat[i].Expand()
            prog.AddLinearConstraint(frombelow.flat[i].Expand() == fromabove.flat[i].Expand())
    

In [14]:
prog = mp.MathematicalProgram()
position = piecewise_polynomial_variable(prog, np.linspace(0, 1, 11), 1, 2)
require_continuity(prog, position)
velocity = position.map(Polynomial.derivative)
require_continuity(prog, velocity)

continuity: (x(0,0) + 0.1 * x(1,0) + 0.01 * x(2,0)) x(0,1)
continuity: (x(0,1) + 0.1 * x(1,1) + 0.01 * x(2,1)) x(0,2)
continuity: (x(0,2) + 0.1 * x(1,2) + 0.01 * x(2,2)) x(0,3)
continuity: (x(0,3) + 0.1 * x(1,3) + 0.01 * x(2,3)) x(0,4)
continuity: (x(0,4) + 0.1 * x(1,4) + 0.01 * x(2,4)) x(0,5)
continuity: (x(0,5) + 0.1 * x(1,5) + 0.01 * x(2,5)) x(0,6)
continuity: (x(0,6) + 0.1 * x(1,6) + 0.01 * x(2,6)) x(0,7)
continuity: (x(0,7) + 0.1 * x(1,7) + 0.01 * x(2,7)) x(0,8)
continuity: (x(0,8) + 0.1 * x(1,8) + 0.01 * x(2,8)) x(0,9)
continuity: (x(1,0) + 0.2 * x(2,0)) x(1,1)
continuity: (x(1,1) + 0.2 * x(2,1)) x(1,2)
continuity: (x(1,2) + 0.2 * x(2,2)) x(1,3)
continuity: (x(1,3) + 0.2 * x(2,3)) x(1,4)
continuity: (x(1,4) + 0.2 * x(2,4)) x(1,5)
continuity: (x(1,5) + 0.2 * x(2,5)) x(1,6)
continuity: (x(1,6) + 0.2 * x(2,6)) x(1,7)
continuity: (x(1,7) + 0.2 * x(2,7)) x(1,8)
continuity: (x(1,8) + 0.2 * x(2,8)) x(1,9)
